In [ ]:
import sys
import os
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime

sys.path.append(os.path.abspath("D:\\burtm\\Visual_studio_code\\PD_related_projects"))

d:\burtm\Visual_studio_code\conda_environments\GeneralPurposeML\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image

# load image from the IAM database
'''url = 'https://fki.tic.heia-fr.ch/static/img/a01-122-02-00.jpg'
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")'''
# Select one word image from the word_images list

#processor = TrOCRProcessor.from_pretrained('microsoft/trocr-small-handwritten')
#processor = TrOCRProcessor.from_pretrained('microsoft/trocr-small-handwritten',force_download=True) #force download clear the cache and redownload the model
#processor = TrOCRProcessor.from_pretrained('microsoft/trocr-small-handwritten',use_fast=False)
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-small-stage1',use_fast=False)

In [4]:
#model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-small-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-small-stage1')

Config of the encoder: <class 'transformers.models.deit.modeling_deit.DeiTModel'> is overwritten by shared encoder config: DeiTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 384,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "model_type": "deit",
  "num_attention_heads": 6,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.48.2"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 384,
  "d_model": 256,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 1024,
  "decode

In [5]:
model=model.encoder

In [7]:
model = model_utils.TruncatedDeiT(model)

In [8]:
print(model)

TruncatedDeiT(
  (embeddings): DeiTEmbeddings(
    (patch_embeddings): DeiTPatchEmbeddings(
      (projection): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ModuleList(
    (0-9): 10 x DeiTLayer(
      (attention): DeiTSdpaAttention(
        (attention): DeiTSdpaSelfAttention(
          (query): Linear(in_features=384, out_features=384, bias=True)
          (key): Linear(in_features=384, out_features=384, bias=True)
          (value): Linear(in_features=384, out_features=384, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (output): DeiTSelfOutput(
          (dense): Linear(in_features=384, out_features=384, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (intermediate): DeiTIntermediate(
        (dense): Linear(in_features=384, out_features=1536, bias=True)
        (intermediate_act_fn): GELUActivation()
      )
      (output): DeiTOutput(


# standard

how much time to fine tune the whole model? 60s per iteration -> 1h per epoch

In [9]:
batch_size=32
N_max=282
input_filename='icdar_train_df_cc_5patches_perName.csv'
transform=processor
huggingface=True

In [10]:
visualization.show_single_image(train_dataloader,index=1000, save_path=None, from_dataloader=True)

NameError: name 'train_dataloader' is not defined

In [11]:
# Define loss function and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device is: ",device)
model = model.to(device)

Device is:  cuda


In [13]:
import pandas as pd
source_path="D:\\burtm\\Visual_studio_code\\PD_related_projects"
train_df = pd.read_csv(f"{source_path}\\outputs\\preprocessed_data\\{input_filename}")
model.eval()

# Initialize a dictionary to store new feature columns
new_features = {}

for index,t in train_df.iterrows():
    image_file = t['file_name']
    x1 = t['x']
    y1 = t['y']
    x2 = t['x2']
    y2 = t['y2']
    image = Image.open(image_file).convert("RGB")
    patch = image.crop((x1, y1, x2, y2))
    if huggingface:
        # the transform is actually an huggingface processor in this case
        inputs = transform(images=patch, return_tensors="pt")
        # Remove batch dimension from inputs
        patch = inputs['pixel_values'].squeeze()
    elif transform:
        patch = transform(patch)
    patch = patch.to(device)
    output = model(patch.unsqueeze(0))
    #cls_token_output = output.last_hidden_state[:, 0, :]  # Extract the CLS token
    cls_token_output = output[:, 0, :] #if i am using a truncated model there is no last_hidden_state argument
    # Add CLS token output dimensions as new columns to the dataframe
    for i, value in enumerate(cls_token_output.squeeze().tolist()):
        column_name = f"f{i+1}"
        if column_name not in new_features:
            new_features[column_name] = []
        new_features[column_name].append(value)
    if index % 100 == 0:
        print(f"Processed {index} images, out of {len(train_df)}")

    
# Add the new features to the DataFrame in one operation
new_features_df = pd.DataFrame(new_features)
train_df = pd.concat([train_df.reset_index(drop=True), new_features_df], axis=1)

Processed 0 images, out of 5640
Processed 100 images, out of 5640
Processed 200 images, out of 5640
Processed 300 images, out of 5640
Processed 400 images, out of 5640
Processed 500 images, out of 5640
Processed 600 images, out of 5640
Processed 700 images, out of 5640
Processed 800 images, out of 5640
Processed 900 images, out of 5640
Processed 1000 images, out of 5640
Processed 1100 images, out of 5640
Processed 1200 images, out of 5640
Processed 1300 images, out of 5640
Processed 1400 images, out of 5640
Processed 1500 images, out of 5640
Processed 1600 images, out of 5640
Processed 1700 images, out of 5640
Processed 1800 images, out of 5640
Processed 1900 images, out of 5640
Processed 2000 images, out of 5640
Processed 2100 images, out of 5640
Processed 2200 images, out of 5640
Processed 2300 images, out of 5640
Processed 2400 images, out of 5640
Processed 2500 images, out of 5640
Processed 2600 images, out of 5640
Processed 2700 images, out of 5640
Processed 2800 images, out of 56

In [ ]:
from utils.dataframes import CustomImageDataset, CustomPatchDataset
from torch.utils.data import Dataset,DataLoader, random_split
source_path="D:\\burtm\\Visual_studio_code\\PD_related_projects"
train_df = pd.read_csv(f"{source_path}\\outputs\\preprocessed_data\\{input_filename}")
model.eval()
dataset = CustomPatchDataset(train_df[train_df['writer']<=N_max] ,
                                        label_column='male', transform=transform, huggingface=huggingface)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
new_features = {}

for idx, batch in enumerate(dataloader):
    patch = batch['image'].to(device)
    output = model(pixel_values=patch)
    cls_token_output = output.last_hidden_state[:, 0, :]  # Extract the CLS token
    for i, value in enumerate(cls_token_output.squeeze().tolist()):
        column_name = f"f{i+1}"
        if column_name not in new_features:
            new_features[column_name] = []
        new_features[column_name].append(value)
    if idx % 100 == 0:
        print(f"Processed {idx * batch_size} images, out of {len(dataset)}")

# Convert the dictionary to a DataFrame
new_features_df = pd.DataFrame(new_features)

Processed 0 images, out of 5640



KeyboardInterrupt



In [14]:
train_df.head()

,writer,same_text,isEng,train,file_name,male,x,y,x2,y2,...,f375,f376,f377,f378,f379,f380,f381,f382,f383,f384
0,190,0,0,1,D:\download\PD project\datasets\ICDAR 2013 - G...,1,988,494,1482,988,...,-0.922357,2.522360,4.304227,-3.425128,3.495620,-4.153519,-0.852054,-1.164748,-2.234782,-2.404248
1,24,0,1,1,D:\download\PD project\datasets\ICDAR 2013 - G...,0,990,495,1485,990,...,-0.485612,1.888572,4.129086,-3.297427,4.332079,-4.687369,-2.287635,-1.905988,-4.285491,-1.362528
2,26,1,1,1,D:\download\PD project\datasets\ICDAR 2013 - G...,1,494,1976,988,2470,...,-1.275970,1.497854,3.086382,-2.358793,4.704427,-5.747137,-3.386876,1.461058,-3.862585,-1.632300
3,190,0,0,1,D:\download\PD project\datasets\ICDAR 2013 - G...,1,494,494,988,988,...,-1.605459,2.483239,4.245601,-3.242137,3.766826,-4.104949,-1.693988,-1.805966,-2.415438,-1.441694
4,160,0,1,1,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1482,494,1976,988,...,-0.307438,1.057468,2.697207,-1.516698,3.344037,-4.181660,-2.132452,-1.470941,-4.696310,-0.665148


In [15]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
file_path=source_path+f"\\outputs\\preprocessed_data\\icdar_EXTRACTED_trocr_stage_1_10blocks_{timestamp}.csv"
train_df.to_csv(file_path, index=False)
print(f"File saved: {file_path}")

File saved: D:\burtm\Visual_studio_code\PD_related_projects\outputs\preprocessed_data\icdar_EXTRACTED_trocr_stage_1_10blocks_20250509_153553.csv


# easy access

In [6]:
def reload_modules():
    import importlib
    import utils.data_loading as data_loading
    import utils.visualization as visualization
    import utils.dataframes as dataframes
    import utils.utils_transforms as u_transforms
    import utils.training_utils as training_utils
    import utils.model_utils as model_utils
    

    importlib.reload(data_loading)
    importlib.reload(visualization)
    importlib.reload(dataframes)
    importlib.reload(u_transforms)
    importlib.reload(model_utils)
    importlib.reload(training_utils)

    return data_loading, visualization, dataframes, u_transforms, training_utils, model_utils
data_loading, visualization, dataframes, u_transforms, training_utils, model_utils = reload_modules()